# Create 3D and 2D images from 4D image/mask files

In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Notebook_imports import *
#from pyforest import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2


from src.utils.Utils_io import Console_and_file_logger, ensure_dir
from src.visualization.Visualize import plot_3d_vol, plot_4d_vol
from src.data.Dataset import get_metadata_maybe, filter_4d_vol, copy_meta_and_save, create_4d_volumes_from_4d_files, create_3d_volumes_from_4d_files, create_2d_slices_from_4d_volume_files, describe_sitk, load_acdc_files, create_2d_slices_from_3d_volume_files 
Console_and_file_logger('3d_Data_generation', logging.INFO)



search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/3d-mri-domain-adaption


2020-11-25 15:39:35,409 INFO -------------------- Start --------------------
2020-11-25 15:39:35,410 INFO Working directory: /mnt/ssd/git/3d-mri-domain-adaption.
2020-11-25 15:39:35,410 INFO Log file: ./logs/3d_Data_generation.log
2020-11-25 15:39:35,410 INFO Log level for console: INFO


In [2]:
# some helpers
def get_4d_img_msk_files(path):
    
    masks = sorted(glob.glob(os.path.join(path, '*mask.nrrd')))
    images = [path.replace('mask', 'clean') for path in masks]
    return images, masks

def get_3d_img_msk_files(path):
   
    masks = sorted(glob.glob(os.path.join(path, '*msk.nrrd')))
    images = [path.replace('msk', 'img') for path in masks]
    return images, masks

def get_4d_img_msk_files_from_split_dir(path):
    masks = sorted(glob.glob(os.path.join(path, 'masks/*.nrrd')))
    images = sorted(glob.glob(os.path.join(path, 'images/*.nrrd')))
    return images, masks

In [4]:
# check import directory
images, masks = get_4d_img_msk_files('/mnt/ssd/data/gcn/gcn_05_2020_ax_sax_86_2nd/AX')
assert(len(images) == len(masks))
len(images)

86

# Create 4D images & masks from 4D images/masks - filter by masked timesteps
needs to be executed per modality !!!

In [9]:
# create 3D volumes from split 4D directory
modality = 'train'
import_path = 'data/raw/GCN_2nd/4D/{}/'.format(modality)
export_path = 'data/raw/GCN_2nd/4D_filtered/{}'.format(modality)
ensure_dir(export_path)

# create a df for monitoring and statistics
df = pd.DataFrame(columns=['images_f', 'masks_f', 'timesteps','img_shape'])

# get all 4d image and mask files
df['images_f'], df['masks_f'] = get_4d_img_msk_files_from_split_dir(import_path)
assert(len(df['images_f']) == len(df['masks_f']))
logging.info('create 4D volumes from: {} with: {} files'.format(import_path, len(df['images_f'])))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_4d_volumes_from_4d_files(img_f, mask_f, export_path) for img_f, mask_f in zip(df['images_f'], df['masks_f'])])

# remember filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '4d_{}_dataset.csv'.format(modality))
df

2020-03-17 20:14:58,345 INFO create 3D volumes from: data/raw/GCN_2nd/4D/train/ with: 159 files
2020-03-17 20:14:58,345 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-02Z1L14N_2006-11-09_volume_clean.nrrd
2020-03-17 20:14:58,353 INFO (14, 10, 192, 138)
2020-03-17 20:14:58,358 INFO filtered timesteps: [2, 5, 7, 11, 13]
2020-03-17 20:14:58,367 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-04NEJQUZ__200-70-31_volume_clean.nrrd
2020-03-17 20:14:58,413 INFO (25, 16, 256, 256)
2020-03-17 20:14:58,437 INFO filtered timesteps: [5, 12, 16, 20, 23]
2020-03-17 20:14:58,487 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-0AE4R74L_1900-01-01_volume_clean.nrrd
2020-03-17 20:14:58,496 INFO (13, 11, 192, 144)
2020-03-17 20:14:58,501 INFO filtered timesteps: [1, 3, 6, 10, 12]
2020-03-17 20:14:58,512 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-0HQQW4ZN_2007-05-23_volume_clean.nrrd
2020-03-17 20:14:58,541 INFO (25, 10, 256, 256)
2020-03-17 20:14:58,552 INFO filt

,images_f,masks_f,timesteps,img_shape
0,data/raw/GCN_2nd/4D/train/images/0000-02Z1L14N...,data/raw/GCN_2nd/4D/train/masks/0000-02Z1L14N_...,"[2, 5, 7, 11, 13]","(5, 10, 192, 138)"
1,data/raw/GCN_2nd/4D/train/images/0000-04NEJQUZ...,data/raw/GCN_2nd/4D/train/masks/0000-04NEJQUZ_...,"[5, 12, 16, 20, 23]","(5, 16, 256, 256)"
2,data/raw/GCN_2nd/4D/train/images/0000-0AE4R74L...,data/raw/GCN_2nd/4D/train/masks/0000-0AE4R74L_...,"[1, 3, 6, 10, 12]","(5, 11, 192, 144)"
3,data/raw/GCN_2nd/4D/train/images/0000-0HQQW4ZN...,data/raw/GCN_2nd/4D/train/masks/0000-0HQQW4ZN_...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)"
4,data/raw/GCN_2nd/4D/train/images/0000-0PTV75MP...,data/raw/GCN_2nd/4D/train/masks/0000-0PTV75MP_...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)"
5,data/raw/GCN_2nd/4D/train/images/0000-0RPELLU8...,data/raw/GCN_2nd/4D/train/masks/0000-0RPELLU8_...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)"
6,data/raw/GCN_2nd/4D/train/images/0000-11YU3CUF...,data/raw/GCN_2nd/4D/train/masks/0000-11YU3CUF_...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)"
7,data/raw/GCN_2nd/4D/train/images/0000-13JLP3HN...,data/raw/GCN_2nd/4D/train/masks/0000-13JLP3HN_...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)"
8,data/raw/GCN_2nd/4D/train/images/0000-1DRHF745...,data/raw/GCN_2nd/4D/train/masks/0000-1DRHF745_...,"[3, 7, 11, 14, 16]","(5, 18, 256, 256)"
9,data/raw/GCN_2nd/4D/train/images/0000-1E8MCV9L...,data/raw/GCN_2nd/4D/train/masks/0000-1E8MCV9L_...,"[0, 4, 9, 11, 15]","(5, 14, 256, 256)"


# Create 3D images & masks from 4D images/masks
needs to be executed per modality !!!

In [6]:
# create 3D volumes from split 4D directory
modality = 'train'
import_path = 'data/raw/GCN_2nd/4D/{}/'.format(modality)
export_path = 'data/raw/GCN_2nd/3D/{}'.format(modality)
ensure_dir(export_path)

# create a df for monitoring and statistics
df = pd.DataFrame(columns=['images_f', 'masks_f', 'timesteps','img_shape'])

# get all 4d image and mask files
df['images_f'], df['masks_f'] = get_4d_img_msk_files_from_split_dir(import_path)
assert(len(df['images_f']) == len(df['masks_f']))
logging.info('create 3D volumes from: {} with: {} files'.format(import_path, len(df['images_f'])))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_3d_volumes_from_4d_files(img_f, mask_f, export_path) for img_f, mask_f in zip(df['images_f'], df['masks_f'])])

# remember filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '3d_{}_dataset.csv'.format(modality))
df

2020-03-17 18:49:03,123 INFO create 3D volumes from: data/raw/GCN_2nd/4D/train/ with: 159 files
2020-03-17 18:49:03,124 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-02Z1L14N_2006-11-09_volume_clean.nrrd
2020-03-17 18:49:03,134 INFO (14, 10, 192, 138)
2020-03-17 18:49:03,139 INFO filtered timesteps: [2, 5, 7, 11, 13]
2020-03-17 18:49:03,153 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-04NEJQUZ__200-70-31_volume_clean.nrrd
2020-03-17 18:49:03,227 INFO (25, 16, 256, 256)
2020-03-17 18:49:03,243 INFO filtered timesteps: [5, 12, 16, 20, 23]
2020-03-17 18:49:03,283 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-0AE4R74L_1900-01-01_volume_clean.nrrd
2020-03-17 18:49:03,294 INFO (13, 11, 192, 144)
2020-03-17 18:49:03,298 INFO filtered timesteps: [1, 3, 6, 10, 12]
2020-03-17 18:49:03,311 INFO process file: data/raw/GCN_2nd/4D/train/images/0000-0HQQW4ZN_2007-05-23_volume_clean.nrrd
2020-03-17 18:49:03,357 INFO (25, 10, 256, 256)
2020-03-17 18:49:03,371 INFO filt

,images_f,masks_f,timesteps,img_shape
0,data/raw/GCN_2nd/4D/train/images/0000-02Z1L14N...,data/raw/GCN_2nd/4D/train/masks/0000-02Z1L14N_...,"[2, 5, 7, 11, 13]","(5, 10, 192, 138)"
1,data/raw/GCN_2nd/4D/train/images/0000-04NEJQUZ...,data/raw/GCN_2nd/4D/train/masks/0000-04NEJQUZ_...,"[5, 12, 16, 20, 23]","(5, 16, 256, 256)"
2,data/raw/GCN_2nd/4D/train/images/0000-0AE4R74L...,data/raw/GCN_2nd/4D/train/masks/0000-0AE4R74L_...,"[1, 3, 6, 10, 12]","(5, 11, 192, 144)"
3,data/raw/GCN_2nd/4D/train/images/0000-0HQQW4ZN...,data/raw/GCN_2nd/4D/train/masks/0000-0HQQW4ZN_...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)"
4,data/raw/GCN_2nd/4D/train/images/0000-0PTV75MP...,data/raw/GCN_2nd/4D/train/masks/0000-0PTV75MP_...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)"
5,data/raw/GCN_2nd/4D/train/images/0000-0RPELLU8...,data/raw/GCN_2nd/4D/train/masks/0000-0RPELLU8_...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)"
6,data/raw/GCN_2nd/4D/train/images/0000-11YU3CUF...,data/raw/GCN_2nd/4D/train/masks/0000-11YU3CUF_...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)"
7,data/raw/GCN_2nd/4D/train/images/0000-13JLP3HN...,data/raw/GCN_2nd/4D/train/masks/0000-13JLP3HN_...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)"
8,data/raw/GCN_2nd/4D/train/images/0000-1DRHF745...,data/raw/GCN_2nd/4D/train/masks/0000-1DRHF745_...,"[3, 7, 11, 14, 16]","(5, 18, 256, 256)"
9,data/raw/GCN_2nd/4D/train/images/0000-1E8MCV9L...,data/raw/GCN_2nd/4D/train/masks/0000-1E8MCV9L_...,"[0, 4, 9, 11, 15]","(5, 14, 256, 256)"


In [9]:
# check exported images
images, masks = get_3d_img_msk_files(export_path)
logging.info(len(images))
logging.info(len(masks))


2019-11-19 01:09:21,909 INFO 765
2019-11-19 01:09:21,910 INFO 765


# Create 2D images & masks from 4D images/masks

In [11]:
# create 2D slices from 4D volumes, change path and modality
path = 'data/raw/peters'
modality = 'all'
only_masked = False # filter timesteps without GT labels, this is necessary during the training
import_path = '{}/{}/{}/'.format(path, '4D', modality)
export_path = '{}/{}_unfiltered/{}/'.format(path, '2D', modality)
ensure_dir(export_path)

# create a df for monitoring and statistics
df = pd.DataFrame(columns=['images_f', 'masks_f', 'timesteps','img_shape'])

# get all 4d image and mask files
df['images_f'], df['masks_f'] = get_4d_img_msk_files_from_split_dir(import_path)
assert(len(df['images_f']) == len(df['masks_f']))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_2d_slices_from_4d_volume_files(img_f, mask_f, export_path, only_masked) for img_f, mask_f in zip(df['images_f'], df['masks_f'])])

# remeber filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '2d_{}_dataset.csv'.format(modality))
df

2020-03-19 17:44:40,672 INFO process file: data/raw/peters/4D/all/images/0000-BC_20161_227_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:41,486 INFO process file: data/raw/peters/4D/all/images/0000-BK_20170_815_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:42,247 INFO process file: data/raw/peters/4D/all/images/0000-BM_20160_419_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:43,140 INFO process file: data/raw/peters/4D/all/images/0000-BR_20170_223_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:43,926 INFO process file: data/raw/peters/4D/all/images/0000-CK_20170_112_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:44,684 INFO process file: data/raw/peters/4D/all/images/0000-CR_20161_208_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:45,381 INFO process file: data/raw/peters/4D/all/images/0000-ES_20161_103_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:46,296 INFO process file: data/raw/peters/4D/all/images/0000-FN_20170_829_-TO-Fu_volume_clean.nrrd
2020-03-19 17:44:46,945 INFO process file: data/raw/peters/4D/all/images

,images_f,masks_f,timesteps,img_shape
0,data/raw/peters/4D/all/images/0000-BC_20161_22...,data/raw/peters/4D/all/masks/0000-BC_20161_227...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 13, 352, 352)"
1,data/raw/peters/4D/all/images/0000-BK_20170_81...,data/raw/peters/4D/all/masks/0000-BK_20170_815...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 13, 336, 336)"
2,data/raw/peters/4D/all/images/0000-BM_20160_41...,data/raw/peters/4D/all/masks/0000-BM_20160_419...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 15, 336, 336)"
3,data/raw/peters/4D/all/images/0000-BR_20170_22...,data/raw/peters/4D/all/masks/0000-BR_20170_223...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 15, 256, 256)"
4,data/raw/peters/4D/all/images/0000-CK_20170_11...,data/raw/peters/4D/all/masks/0000-CK_20170_112...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 12, 352, 352)"
5,data/raw/peters/4D/all/images/0000-CR_20161_20...,data/raw/peters/4D/all/masks/0000-CR_20161_208...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 13, 256, 256)"
6,data/raw/peters/4D/all/images/0000-ES_20161_10...,data/raw/peters/4D/all/masks/0000-ES_20161_103...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 15, 352, 352)"
7,data/raw/peters/4D/all/images/0000-FN_20170_82...,data/raw/peters/4D/all/masks/0000-FN_20170_829...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 11, 336, 336)"
8,data/raw/peters/4D/all/images/0000-FR_20161_12...,data/raw/peters/4D/all/masks/0000-FR_20161_129...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 13, 256, 256)"
9,data/raw/peters/4D/all/images/0000-HT_20170_22...,data/raw/peters/4D/all/masks/0000-HT_20170_220...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(30, 15, 256, 256)"


## Create 2D images/masks from 3D ACDC files

In [2]:
# create 2D slices from 4D volumes
import_path = 'data/raw/ACDC/original/val'
export_path = 'data/raw/ACDC/2D/val'
ensure_dir(export_path)

# create a df for monitoring and statistics
df = pd.DataFrame(columns=['images_f', 'masks_f', 'timesteps','img_shape'])

# get all 3d image and mask files
df['images_f'], df['masks_f'] = load_acdc_files(import_path)
assert(len(df['images_f']) == len(df['masks_f']))

# slice all 3D volumes, copy metadata and save them
steps, shapes = zip(*[create_2d_slices_from_3d_volume_files(img_f, mask_f, export_path) for img_f, mask_f in zip(df['images_f'], df['masks_f'])])

# remeber filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '2d_test_dataset.csv')
df

2019-09-20 17:46:26,016 INFO process file: data/raw/ACDC/original/val/patient016/patient016_frame01.nii.gz
2019-09-20 17:46:26,060 INFO process file: data/raw/ACDC/original/val/patient016/patient016_frame12.nii.gz
2019-09-20 17:46:26,080 INFO process file: data/raw/ACDC/original/val/patient017/patient017_frame01.nii.gz
2019-09-20 17:46:26,099 INFO process file: data/raw/ACDC/original/val/patient017/patient017_frame09.nii.gz
2019-09-20 17:46:26,122 INFO process file: data/raw/ACDC/original/val/patient018/patient018_frame01.nii.gz
2019-09-20 17:46:26,144 INFO process file: data/raw/ACDC/original/val/patient018/patient018_frame10.nii.gz
2019-09-20 17:46:26,168 INFO process file: data/raw/ACDC/original/val/patient019/patient019_frame01.nii.gz
2019-09-20 17:46:26,251 INFO process file: data/raw/ACDC/original/val/patient019/patient019_frame11.nii.gz
2019-09-20 17:46:26,301 INFO process file: data/raw/ACDC/original/val/patient020/patient020_frame01.nii.gz
2019-09-20 17:46:26,336 INFO process 

,images_f,masks_f,timesteps,img_shape
0,data/raw/ACDC/original/val/patient016/patient0...,data/raw/ACDC/original/val/patient016/patient0...,01,"(10, 256, 192)"
1,data/raw/ACDC/original/val/patient016/patient0...,data/raw/ACDC/original/val/patient016/patient0...,12,"(10, 256, 192)"
2,data/raw/ACDC/original/val/patient017/patient0...,data/raw/ACDC/original/val/patient017/patient0...,01,"(9, 256, 216)"
3,data/raw/ACDC/original/val/patient017/patient0...,data/raw/ACDC/original/val/patient017/patient0...,09,"(9, 256, 216)"
4,data/raw/ACDC/original/val/patient018/patient0...,data/raw/ACDC/original/val/patient018/patient0...,01,"(8, 256, 216)"
5,data/raw/ACDC/original/val/patient018/patient0...,data/raw/ACDC/original/val/patient018/patient0...,10,"(8, 256, 216)"
6,data/raw/ACDC/original/val/patient019/patient0...,data/raw/ACDC/original/val/patient019/patient0...,01,"(11, 256, 216)"
7,data/raw/ACDC/original/val/patient019/patient0...,data/raw/ACDC/original/val/patient019/patient0...,11,"(11, 256, 216)"
8,data/raw/ACDC/original/val/patient020/patient0...,data/raw/ACDC/original/val/patient020/patient0...,01,"(8, 256, 208)"
9,data/raw/ACDC/original/val/patient020/patient0...,data/raw/ACDC/original/val/patient020/patient0...,11,"(8, 256, 208)"
